# GARCH Volatility Analysis - DOT/USD

**Analysis Date**: 2025-10-07

## Overview
This notebook presents comprehensive GARCH volatility analysis.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
import sys
import os

# Setup plotting
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
%matplotlib inline

## 1. Load Processed Data

In [ ]:
# Load processed returns data
data_path = Path("data/processed/2025-10-07/DOT_USD/processed.csv")
returns_df = pd.read_csv(data_path, index_col='timestamp', parse_dates=True)
print(f"Loaded {len(returns_df)} records for DOT/USD")
returns_df[['close', 'log_returns']].head()

## 2. Returns Analysis

In [ ]:
# Basic returns statistics
returns_stats = returns_df['log_returns'].describe()
returns_stats['skewness'] = returns_df['log_returns'].skew()
returns_stats['kurtosis'] = returns_df['log_returns'].kurtosis()
returns_stats

In [ ]:
# Plot returns distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Time series
ax1.plot(returns_df.index, returns_df['log_returns'], alpha=0.7, linewidth=0.8)
ax1.set_title(f'{asset} - Log Returns')
ax1.set_ylabel('Returns')
ax1.grid(True, alpha=0.3)

# Distribution
ax2.hist(returns_df['log_returns'], bins=50, alpha=0.7, density=True)
ax2.set_title(f'{asset} - Returns Distribution')
ax2.set_xlabel('Returns')
ax2.set_ylabel('Density')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. GARCH Model Results

In [ ]:
# Load GARCH comparison results
comp_path = Path("results/2025-10-07/comparison/garch_model_comparison.csv")
comparison_df = pd.read_csv(comp_path)
asset_comparison = comparison_df[comparison_df['asset'] == 'DOT/USD']
asset_comparison

In [ ]:
# Visualize model comparison
fig, ax = plt.subplots(figsize=(10, 6))
models = asset_comparison['model']
aic_values = asset_comparison['aic']

bars = ax.bar(models, aic_values, color=['skyblue', 'lightcoral', 'lightgreen'])
ax.set_title(f'{asset} - GARCH Model Comparison (AIC)')
ax.set_ylabel('AIC (Lower is Better)')
ax.grid(True, alpha=0.3)

# Add value labels on bars
for bar, aic in zip(bars, aic_values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{aic:.1f}', ha='center', va='bottom')

plt.show()

## 4. Best Model Parameters

In [ ]:
# Get best model
best_model_row = asset_comparison.loc[asset_comparison['aic'].idxmin()]
best_model_name = best_model_row['model']
best_model_params = best_model_row['params']

print(f"Best Model: {best_model_name}")
print(f"AIC: {best_model_row['aic']:.1f}")
print(f"Log Likelihood: {best_model_row['log_likelihood']:.1f}")
print(f"Persistence: {best_model_row['persistence']:.3f}")

print("\nModel Parameters:")
if isinstance(best_model_params, str):
    best_model_params = eval(best_model_params)  # Convert string to dict
for param, value in best_model_params.items():
    print(f"  {param}: {value:.6f}")

## 5. Volatility Analysis

In [ ]:
# Display volatility plot
from IPython.display import Image

plot_path = Path("results/2025-10-07/DOT_USD/plots/volatility_analysis.png")
if plot_path.exists():
    display(Image(filename=str(plot_path)))
else:
    print("Volatility plot not found")

## 6. Trading Insights

In [ ]:
# Generate trading insights
persistence = best_model_row['persistence']
if not pd.isna(persistence):
    if persistence > 0.95:
        persistence_insight = "High persistence - volatility shocks have long-lasting effects"
    elif persistence > 0.8:
        persistence_insight = "Moderate persistence - volatility mean-reverts moderately"
    else:
        persistence_insight = "Low persistence - volatility mean-reverts quickly"

volatility = returns_df['log_returns'].std()
if volatility > 0.01:
    vol_insight = "High volatility - suitable for volatility trading strategies"
elif volatility > 0.005:
    vol_insight = "Moderate volatility - balanced risk-return profile"
else:
    vol_insight = "Low volatility - stable but lower potential returns"

print("Trading Insights:")
print(f"- {{persistence_insight}}")
print(f"- {{vol_insight}}")
print(f"- Best model type: {{best_model_name}}")
print(f"- Recommended: Use for {{'shorter-term' if persistence < 0.8 else 'longer-term'}} volatility strategies")

## 7. Conclusion


### Summary for DOT/USD

- **Best Model**: {{best_model_name}}
- **Volatility Persistence**: {{persistence:.3f}}
- **Returns Volatility**: {{volatility:.4f}}
- **Model Fit Quality**: {{'Excellent' if best_model_row['aic'] < 1000 else 'Good'}}

### Recommended Actions:
1. Monitor volatility regimes using the {{best_model_name}} model
2. Adjust position sizing based on volatility forecasts
3. Consider volatility-based entry/exit signals
